# Recurrent Neural Network in 2D and 3D lightcones

There are several ideas we would like to explore, concerning RNN networks.

1. Compessing spacial data to N parameters with CNN, with network weights independent of time (redshift) and feeding those to RNN network which in every step tries to predict astrophysical parameters, and then weight somehow the sum all of those in the loss function.
2. Using the whole CNN as a part of RNN network which updates it's weights in each iteration
3. Some hybrid of usual convolution in spacial dimensions and WaveNet-like convolutions in time dimension

In 1. and 2. for RNN part, we use LSTM cells

In [16]:
import numpy as np
import sys
import os
import json
import tensorflow as tf
# os.environ['KERAS_BACKEND'] = 'tensorflow'


# #setting up GPU
config = tf.compat.v1.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 1. #setting the percentage of GPU usage
config.gpu_options.visible_device_list = "0" #for picking only some devices
# config.gpu_options.allow_growth = True

#passing tf session to keras!
# from keras.backend.tensorflow_backend import set_session
tf.compat.v1.keras.backend.set_session(tf.compat.v1.Session(config=config)) 

# from keras import backend as K
tf.keras.backend.set_image_data_format('channels_last')

In [18]:
#working with sliced data shape (10000, 4, 25, 25, ~500)
DataLoc = "../data/"
XName = "data3D_boxcar444_sliced22_float32.npy"
YName = "databaseParams_float32.npy"
YbackupName = "databaseParams_min_max.txt"

In [34]:
Y = np.load(DataLoc+YName)
print(Y.shape)
with open(DataLoc+YbackupName, "r") as f:
    Params = json.load(f)
print(json.dumps(Params, indent=4))
Params["min"] = np.array(Params["min"], dtype = np.float32)
Params["max"] = np.array(Params["max"], dtype = np.float32)

Y = (Y - Params["min"]) / (Params["max"] - Params["min"])
shape = Y.shape
Y = Y[..., np.newaxis]
Y = np.broadcast_to(Y, shape + (4,))
Y = np.swapaxes(Y, -1, -2)
Y.shape
Y[23, :, 2]

(10000, 4)
{
    "parameters": [
        "ZETA",
        "TVIR_MIN",
        "L_X",
        "NU_X_THRESH"
    ],
    "min": [
        10.023063659667969,
        4.000161170959473,
        38.00041961669922,
        100.13026428222656
    ],
    "max": [
        249.9544677734375,
        5.998933792114258,
        41.99989700317383,
        1499.37109375
    ]
}


array([0.92404807, 0.92404807, 0.92404807, 0.92404807], dtype=float32)

In [35]:
X = np.load(DataLoc+XName)

In [43]:
X = np.swapaxes(X, 2, 4)
X = X.reshape(-1, X.shape[-3], X.shape[-2], X.shape[-1])
Y = Y.reshape(-1, Y.shape[-1])
print(X.shape)
print(Y.shape)

(40000, 4)

In [56]:
from utilities3 import basicTDT
Xtrain, Ytrain, Xval, Yval, Xtest, Ytest = basicTDT(X, Y, 0.8, 0.1, 0.1)

MemoryError: Unable to allocate array with shape (32000, 526, 25, 25) and data type float32

In [57]:
from sklearn.model_selection import train_test_split
RState = np.random.RandomState(seed=1312)
Xtrain, Xtest, Ytrain, Ytest = train_test_split(X, Y, test_size=0.1, random_state=RState)

MemoryError: Unable to allocate array with shape (40000, 526, 25, 25) and data type float32

In [51]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, TimeDistributed, Dense
from tensorflow.keras.layers import Conv3D, Conv2D, Conv1D
from tensorflow.keras.layers import ConvLSTM2D
from tensorflow.keras.layers import BatchNormalization

In [ ]:
class HyperParams1:
    def __init__(
        LossDecay = 0.9,
        LossLength = 10,
        ):
        self.LossDecay = LossDecay
        self.LossLength = LossLength
        
model1 = Sequential()
model1.add

    